In this tutorial, we will write a Path Integral Code from scratch in python.  
At the end of this tutorial, we will have a simple path integral code that will be 
able to calculate properties of two (unpolarized) electrons in a quantum dot 
(which we will model as a double quantum harmonic well.)

Throughout this tutorial, we will work in atomic units (i.e. hartree and bohr radii).  The relevant parameters we will work with give 
$\lambda=\frac{\hbar^2}{2m}=0.5$ for electrons and inverse temperature $\beta = \frac{1}{kT}$ (k=1.0 in our units).

### Contents

* Preliminaries
* Formalism
    * The potential action
    * The kinetic action
* Building code pieces
    * Path Object
    * The Kinetic Action
    * Defining Vext
    * Potential Action
    * Your first PIMC Code
* Our First Move: Single Slice Displacements
    * Choosing new coordinates
    * Evaluating the acceptance probability
    * Putting it together
        * Kinetic Test
        * External Potential Test
    * Visualizing the paths
* Our second move: Displace
    * Potential Test, take 2
* Our third  move: Lowering the temperature and the Bisection Move
* The final steps
    * Observables
    * Correlation: The interaction potential
    * Adding the double well
    * Manipulating a quantum dot

# Goals
    Goal 1: Calculate the kinetic action of a path
    Goal 2: Calculate the potential action of a path
    Goal 3: Implement Monte Carlo moves to sample paths from the density matrix
    Goal 4: Improve Monte Carlo moves to sample paths more efficiently

## Formalism  
The many-particle density matrix is defined as 
$$\rho(R,R';\beta) = \langle R | exp[-\beta \hat{H} ] |R'\rangle$$,
where $R$ is a $3N$-dimensional vector representing the positions of all $N$ particles in the system.  The density matrix has the property that the thermal average of any observable quantity can be expressed as 
$$\langle \hat{O} \rangle = \frac{1}{Z} \int dR dR' \rho(R,R';\beta)O(R,R')$$
where $$Z= \int dR \rho(R,R;\beta)$$


Many properties we are interested in (the energy, pair correlation functions, etc.) are 
*diagonal* in position space.  This allows us to compute properties from the diagonal of the density matrix, $\rho(R,R;\beta)$.  

In PIMC, we expand the diagonal density matrix by dividing the exponential into $M$ pieces and inserting the resolution fo unity in position space between each piece, yielding
$$\rho(R_0,R_0;\beta)=\int dR_1 ... dR_{M-1} \rho(R_0,R_1;\tau)\rho(R_1,R_2;\tau)...\rho(R_{M-1},R_0;\tau)$$ where $\tau\equiv\beta/M$. Note that each density matrix $\rho(R_i,R_{i+1};\tau)$, in some sense, "connects" particle positions at *time slice* $i$ to the adjacent slice at slice $i+1.$  Thus, we can think of the progression of particle positions from time slice to time slice as a discrete $path.$

Haven't we just made more work for ourselves, increasing the dimensions over which we have to integrate?  In some sense, we have, but in Monte Carlo integration, the computational demand does not increase rapidlty with dimensionality as in standard numerical quadrature.  For small $\tau$ (high temperature), we can write down very good approximations to $\rho(R,R';\tau).$  Physically, this is because particle motion is uncorrelated at very high temperature. By using Metropolis Monte Carlo to evaluate the above intergral, we can then sample the desnity matrix at any lower temperature we want by choosing the number of time-slices, $M.$  

We can then compute thermal averages by writing
$$\langle O \rangle_\textrm{thermal} = \int dR_0..dR_{M-1} \times \\ \frac{1}{Z}\rho(R_0,R_1;\tau)\rho(R_1,R_2;tau)...\rho(R_{M-1},R_0;\tau) \frac{(\hat{O} \rho)(R_0,R_1;\tau)}{ \rho(R_0,R_1;\tau)}$$

where the sampling probability is the $\rho...\rho$ piece and the value to average is the $(\hat{O}\rho)/\rho$ piece.   Let us label the sampling probability $\pi(\{R_i\})$ where $\{R_i\}$ represents a particular set of *paths* for the particles.  Let $W_O$ represent the value to be averaged for that path.  Note that the first position in the path $R_0$ is also the last position in the path.  This indicates that the path closes upon itself resulting in objects that resembles ring polymers.  

![title](figures/test_beads.png)

The entire PIMC method can then be summarized by the following algorithm:   

* Move the path variables $\{R_i\}$ such that path configurations are sampled with probability $\pi(\{R_i\})$ given above.
* For each path configuration and each observable, $\hat{O}$, compute $W_O(\{R_i\})$ and add to the running average.
* Repeat until averages have sufficiently small error bars.

It turns out that it is much easier to work with the the logarithm of the short time density matrix instead of the density matrix itself  In particular, define the action $S$, as 

$$S(R,R';\tau) = -\ln[\rho(R,R';\tau)$$

We divide this action into two pieces: one resulting from the potential energy and one from the kinetic energy.

### The potential action
We will work in a simple approximation for the potential action known as the *primitive* approximation.  In this approximation, the potential action is given simply $\tau$ time the potential energy at each time slice

$$V(R,R';\tau)= \frac{\tau}{2}\left[V(R)+V(R')\right]$$

In our two-electron problem, we will have an external potential representing the confining field of the dot and a modified Coulomb interaction between the particles.



### The kinetic action
The kinetic energy operator for a single particle is simply $\hat{T} = -\lambda \nabla^2$, where $\lambda=\hbar^2/2m$.  We can analytically work out that, in three dimensions. 

$$T(r,r';\tau) = -\frac{3}{2}\ln(4\pi\lambda\tau) + \frac{|r-r'|^2}{4\lambda\tau}$$

Note that we have used small $r$ and $r'$ to denote a single particle at two adjacent time slices.  Written this way, the kinetic action looks like a spring potential acting between the time slices. 

Combining our kinetic and potential actions, we can write down our complete action for a single link as 
$$S(R,R';\tau) = \frac{\tau}{2}\left[ V_\textrm{ext}(r_1)+V_\textrm{ext}(r_2)+V_\textrm{ee}(|r_1-r_2|) + V_\textrm{ext}(r_1')+V_\textrm{ext}(r_2')+V_\textrm{ee}(|r_1'-r_2'|) + \frac{|r_1-r_1'|^2}{4\lambda\tau} + \frac{|r_2-r_2'|^2}{4\lambda\tau} - 3\ln(4\pi\lambda\tau)\right]$$

### Path Object

A configuration of the electron paths in the system will be represented by a three-dimensional numpy array. We will group array with other data and functions into a Path object.  A single particle at a time slice is sometimes called a *bead*.  For this reason, the path array for an `Path` instance `path` will be called `path.beads`.  The path variables are indexed as

```python
Path.beads[islice,iptcl,idim]
```

where the first dimension is the slice index in imaginary time, the second is the particle index: `iptcl` is 0,1, the third is spatial dimension: `idim` is 0,1,2, representing the x,y, or z, coordinate.  Each slice is connected to the next, with the last slice also connected to the first. 

*Note:* Inside `Path` class function definitions, you will access the sam array through `self.beads`.

The Path class is defined in `path.py`. Let us take a quick look at the structure of our object.  To begin with, there is an `def__init__` function that is called whenever our class is initialized and sets up the class.  It defines the number of particles, time slices, as well as the time step and lambda (`lambda` is a reserved keyword in Python, so we use `lam` instead).  There is also a `def RelabelBeads` which we will explain below.  The rest of the class currently consists of pieces that you are going to fill in like the actions and the energies. 

To create a Path object instance we can call
``` Python
nslice = 5
nptcl  = 2
ndim   = 3
test_beads = np.loadtxt(fname).reshape(nslice,nptcl,ndim)
tau = lam = 0.5
path = Path(test_beads,tau,lam)
```
which will load a path saved in `data/TestPath.dat`. The above lines are saved in `path.py::load_test_path` for your convenience

In [1]:
import numpy as np
import path as pfuncs

path = pfuncs.load_test_path(str_rep=True)
path.SetPotential(lambda x:0.0)
path.SetCouplingConstant(0.0)

print( "slice 0 of the particle 1 in the z dimension is %14.12f. "%path.beads[0,1,2] )

path nptcl = 2, inverse temperature beta = 0.4000
slice 0 of the particle 1 in the z dimension is -0.566223936211. 


You should find that the slice is at -0.566223936211.

*A note about RelabelBeads:*

Our path integrals are periodic in imaginary time and it doesn't matter which "slice" we call slice 0.   To simpliy our life, we are going to add a function to our class that relabels the slice labels so the one we call "slice 0" is selected randomly.  This way, we can always move what is currently "slice 1, but we will still eventually move all the slices.  This saves us from worrying about wrapping around from the last slice to the first slice in all our functions.  There is nothing physical to this and it is simply a technical book-keeping apparatus.  Toward that end, we have the function `RelabelBeads` in `Path.`

#### The Kinetic Action

    
We need to evaluate both the Kinetic Action and the Potential Action.  The Kinetic Action is of the form 

$$\sum_i -\ln\left[\langle R(t) | \exp[-\tau \lambda \nabla^2]R(t+\tau)\rangle \right] = \textrm{const}+\sum_{t,i} \frac{|r_{i,t}-r_{i,t+\tau}|^2}{4\lambda\tau}$$

Since the constant will not change when a path is moved, we need not evaluate it when we decide whether to accept or reject a move.

Let's start out by modifying the function to our `Path` of the form `def KineticAction(self,slice1,slice2):
`

The indices of the slices are passed into this function as slice1 and slice2.

1.  You can get the dot product of two vectors by calling `numpy.dot`
2.  You can get $\lambda \tau$ by calling `self.lam * self.tau`

To test our function, we can add to our initialization

In [2]:
reload(pfuncs)
path = pfuncs.load_test_path()
path.SetPotential(lambda x:0.0)
path.SetCouplingConstant(0.0)
print "The value of the kinetic action is ", path.KineticAction(1,2)

The value of the kinetic action is  0.567148772825


You should get a result of 0.567148772825.

#### Defining Vext

We will now define a function for the $V_\textrm{ext}$.  Our `Path` object takes a function (which we are about to write) as a paramter.  In python, functions are first-class objects (https://en.wikipedia.org/wiki/First-class_function).  This means that you are allowed to pass around functions as arguments to functions which can then be used.  We are now going to write a function that takes a single coordinate and returns the values of the harmonic potential

$$V_\textrm{ext}(r)=\frac{1}{2}m\omega^2|r|^2$$

where we will set $m=\omega=1.$  Later in this tutorial, we will generalize this to a double well quantum dot.  Let's start by writing the function.



In [3]:
def HarmonicOscillator(pos,mass=1.0,omega=1.0):
    # --- solution ---- 
    r2  = (pos*pos).sum() # sum over particles and dimensions
    pot = 0.5*mass*omega**2.*r2
    # --- solution ---- 
    return pot
#return the harmonic oscillator
# potential for a single particle at position r1

Let us now test this function by using

In [4]:
reload(pfuncs)
path = pfuncs.load_test_path()
path.SetPotential(HarmonicOscillator)
path.SetCouplingConstant(0.0)
print "The value of the external potential is", path.Vext(np.array([0.1,0.3,0.1]))

The value of the external potential is 0.055


You should get the answer 0.055.

#### Potential Action

We are using the primitive approximation to the potential.  This means that it is of the form $\tau V.$  Fill in the potential action `PotentialAction(self,slice1,slice2):` that is currently part of our object. We will test the potential action by calling


In [5]:
reload(pfuncs)
path = pfuncs.load_test_path()
path.SetPotential(HarmonicOscillator)
path.SetCouplingConstant(0.0)
print "The value of the potential action between slice 1 and 2 (with a harmonic external potential is)", path.PotentialAction(1,2)

The value of the potential action between slice 1 and 2 (with a harmonic external potential is) 2.46414551377


The answer you expect is 2.46414551377

### Your First PIMC Code

We are now going to go ahead and actually write our first PIMC code. This code will mainly
be a big loop over two functions: a move you will write soon that implements a monte carlo
step and a call to calculate the total energy of your system.

It will look like:

In [5]:
def pimc(nstep,path,move_list,block_size=10):
    nblock  = nstep//block_size
    nmove   = len(move_list)

    nslice,nptcl,ndim = path.beads.shape
    path_trace = np.zeros([nblock,nslice,nptcl,ndim])
    etrace  = np.zeros(nblock) # energy trace
    naccept = np.zeros(nmove,dtype=int) # number of accepted moves, resolved for each move
    iblock = 0
    for istep in range(nstep):
        for imove in range(len(move_list)):
            if (move_list[imove](path)):
                naccept[imove] += 1
            if istep%block_size == 0:
                etrace[iblock] = path.Energy()
                path_trace[iblock] = path.beads.copy()
                iblock += 1
    return naccept,etrace,path_trace

You will also need to add a `SingleSliceMove` which you will fill in below.

In [10]:
def SingleSliceMove(Path):
    Path.RelabelBeads()
    #add your things here
    #make sure to remember if you reject the move, to restore the old location of the coordinates
    return 0.0 #accepted # return true if accepted, otherwise return false

Note that in the above section of code, the energy is only evaluated every `block_size`
steps (in this case 10). This is simply because evaluating the energy is computationally
expensive, and it does not help to evaluate an observable much more often than every 10 steps (recall that is the autocorrelation time).

Of course, all the work is in writing the move that implements the monte carlo step. Let's do that now

#### Our First Move: Single Slice Displacements

We would now like to implement a single step of the Monte Carlo algorithm.

Your move should move (or not if you reject) the appropriate bead and make sure that it
updates the accepted and attempted number of moves.

The call to `Path.RelabelBeads()` relabels the time slices so the loop begins in a random starting place (this is done so we can always move slice 1).

This function must implement a single step of the Monte Carlo algorithm.  Let's recall how a Monte Carlo algorithm works again:

1.  Choose new coordinates for your system

2. Evaluate the acceptance probability and accept if the ratio is greater then a random number uniformly chosen in $(0,1)$

Let's look at each of these pieces in turn:

#### Choosing new coordinates 

Different moves will change the coordinates in different ways.  For this move, we will change the coordinates of "slice 1" (we start counting from "slice 0") of a random particle (which you select).  Without loss of generality, we can choose "slice 1" since we have randomly rotated the loops at the beginning of this move.

Then we will choose a new place fo rthis coordinate by uniformly displacing it by an amount chosen uniformly at random. Remember to store what and where you've moved things because you will have to move it back to its old location if you reject the move.

A useful function call to know if that `numpy.random.random(3)` will give you a numpy array of 3 elements that are between 0 and 1.  Also, if you have a numpy array, you can multiply and add by a scalar as you would naturally expect.



#### Evaluating the acceptance probability

In metropolis markov chain Monte Carlo, the probability of accepting is 

$$\exp[-S_\textrm{new}+S_\textrm{old}] \frac{T(\textrm{new}\rightarrow \textrm{old})}{T(\textrm{old}\rightarrow \textrm{new}]}$$

The first term is the exponential of the actions.  The actions consists of the kinetic and potential actions (part of which we won't implement till later.  Just use it as 0 for now).  You need to evaluate the dfference between the old and new versions of the action.  Although you could evaluate the total old action and the total new action, this would be a bad idea and result in a very slow code.  instead just evalaute the difference in the pieces of the path that you have changed.

Useful things to keep in mind:

1.  It might be easier to work with log of probabilities instead of the probablities themsleves.
2.  Change one bead changes two links in the kinetic action.

The second term,
$$\frac{T(\textrm{new}\rightarrow \textrm{old})}{T(\textrm{old}\rightarrow \textrm{new}]}$$

is the probability that you've selected the current new configuration over the probability that you would return to the old configuration given that you started in the new configuration.  Because we are simply moving something uniformly with a box, this ration is going to be 1.0 for this move.




#### Putting it together

You should proceed to write this function by implementing these two pieces. Here are a copule things to keep note of:

1.  you will first evaluate the old action on what you are about to move, move it, and then evaluate the new action.

2.  Remember that if you reject to resotre the pieces you've changed (which means you have to save it beforehand).

3.  Remember, when you move the "bead 1", you will have to compute the kinetic action both from bead 1 to 2 and from bead 0 to 1.

Once you have finished writing your function, we need to come up wiht a way to test it.

#### Kinetic Test

In testing monte carlo codes, it's very important that we test things as incrementallly as we can.  Toward that end, let's start by testing things with the external (and interparticle) potentials to be set to 0.  We can do this by calling



In [6]:
from pimc import SingleSliceMove

In [11]:
reload(pfuncs)
tau = 0.1
lam = 0.5

nslice = 5
nptcl  = 2
ndim   = 3
path = pfuncs.Path(np.random.randn(nslice,nptcl,ndim),tau,lam)
path.SetPotential(lambda x:0.0)
path.SetCouplingConstant(0.0)

nstep = 2000
move_list = [SingleSliceMove]
naccept,etrace,path_trace = pimc(nstep,path,move_list)
nmove = len(move_list)*nstep
acceptance_rate = float(naccept.sum())/nmove
print('acceptance rate: {acc:4.2f}%'.format(acc=acceptance_rate*100.) )

acceptance rate: 60.55%


To analyze the reults, copy CalcStatistics.py from the website:  http://clark.physics.illinois.edu/CalcStatistics.py and put it in the folder you are working with. 

In [13]:
from CalcStatistics import Stats
nequil = 50
Stats(etrace[nequil:])

(2.4280885636954901, 0.10696777152928925, 5.0037970552091826)

In [15]:
beta = tau*nslice
3.*beta/2.

0.75

When debugging your code, you should first run with a few number of steps (<5000) and
then once it appears to be working you should run it with more steps (~50000) to lower the
error bars.

Now we will be working with two non-interacting particles. Let's now figure out what answer
you should expect. Because there is no length scale in the problem for the polymer to
"compare" itself too, it can't distinguish a quantum world from a classical world.
Therefore, you expect the energy to reflect the classical energy of a particle at this
temperature. By the equipartition theorem, this would be $\frac{1}{2} k_B T$ per degree of freedom.

Compare your answer to this and verify that it agrees.

Don't continue until you can succesfully get the kinetic energy.  Also at this point, you should note what the acceptance ratio of your move is (in a little bit, we will write a better move that has a much better acceptance ratio)

#### External Potential Test:

Now, let's turn the external potential back on and see if we can reproduce the correct
result for 2 quantum particles in a harmonic well. To do this we make the same calls with a
different function sent to our class:
TestSingleSlicePotential




In [16]:
reload(pfuncs)
path=pfuncs.Path(np.zeros((numTimeSlices,numParticles,3),float),tau,lam)
path.SetPotential(HarmonicOscillator)
path.SetCouplingConstant(0.0)
moveList = [SingleSliceMove]
print pimc(numSteps,path,moveList,"SingleSlicePotential")

TypeError: unsupported operand type(s) for //: 'int' and 'str'

Because this is a harmonic oscillator, we should be able to calculate the energy
analytically for any temperature. In order to do this, you can calculate all the
eigenstates of the harmonic oscillator and then simply occupy them with the probability
Add a function to your code like

In [ ]:
def HarmonicEnergy(tau,numTimeSlices,numParticles):
    # none
    return 0.0

that computes the harmonic oscillator answer.

Run your system and compare the analytic results with those calculated exactly. Note: you
may notice that you are getting the incorrect answer! Let's try to understand why this is
the case.

Note: Currently skipping visualization

#### Our second move: Displace

To increase efficiency, we will add another type of move to our simulation. While the
single-slice move does a reasonable job changing the shape of the path, it fails to move
the center of mass very quickly. Fortunately, a very simple move exists which complements
this move. The "displace" move rigidly translates an entire electron path by some random
displacement vector. Please implement the move below

In [ ]:
def DisplaceMove(Path):
    # First, compute the total potential action for all time slices.
    # This move won't change the kinetic action, so you don't need to compute it
    # Don't forget the link action from the last slice back to the first!
    oldV = 0.0
    # Save a copy of the old path
    savePath = Path.beads.copy()
    # Now, create a random vector for displacement
    delta = 4.0*numpy.array([random.random()-0.5, random.random()-0.5, random.random()-0.5])
    # move all the time slices
    # Compute the new potential action
    newV = 0.0
    # Accept or reject based on the change in potential action
    # Remember to copy back savePath if you reject
    # Remember to return True if you accept and False if you reject

that shifts all the slices of the particle by some amount delta randomly selected from a
box. Notice that when you do this rigid shift, none of the spring lengths will change
(since the imaginary time slices don't change with respect to each other). Therefore, you
just need to calculate the potential action. Remember that our loop is closed so there is a
closed link between slice NumTimeSlices-1 and slice 0.

#### Potential Test: Take 2

Before we noticed when running our simulation that:
1. the error bars were large
2. the autocorrelation time that was calculated was large
3. the autocorrelation time as estimated from looking at our energy trace was large
4. the visualized paths moved very slowly.

Before we noticed when running our simulation that:

1. the error bars were large
2. the autocorrelation time that was calculated was large
3. the autocorrelation time as estimated from looking at our energy trace was large
4. the visualized paths moved very slowly.

We will now do the same simulation (with the same number of steps) but include a displace
move. To test this move, run [something that is currently missing]

Notice, this time, that the error was much smaller (and gets the correct answer) and the autocorrelation time much better. We will visualize our system again by calling

Go ahead and visualize your paths again. 

You should notice a major difference showing that between steps the paths move significantly further.

#### Our third move: Lowering the temperature and the BisectionMove

We now have a Monte Carlo code that works reasonably well at high temperatures. We are now
going to lower our temperature by including 20 time slices. To test our system, run
below

In [ ]:
#still coming


You should note the following things. To begin with, notice that
1. the autocorrelation time is large
2. the error bars are large.

This problem is due to the fact that we are moving only a single slice at a time. Since
each "bead" in the path is connected to its neighbors through the kinetic action, each
single-slice move cannot move the bead very far. (Imagine holding hands in a ring of 100
people, and each person, one at a time, takes a turn taking a step in a random direction.)
To improve the efficiency, it is better to move a continuous section of time slices
simultaneously. We are going to try to remedy that now. Our new move will move 7 slices
(trivially slices at a time)
We have supplied this move. It is called Bisection and we will call it with the same calls
instead replacing the call to PIMC with

In [ ]:
PIMC(numSteps,Path,Bisection,"Bisection"

To appreciate the difference between these two moves, compare the error bars and
autocorrelation time when the system is equilibrated with SingleSlice move as compared with
Bisection. (Of course, this latter move is somewhat slower but you still normally win out
significantly in the end)
Although the difference is certainly noticeable with the 20 slices that we have been using,
it is especially important as we increase quantum effects (for example with 50 slices the
SingleSliceMove becomes unusable)

### The final steps

#### Observables

Beyond the energy, we would also like to know where our electrons are in our system as well
as the correlations between them. This will become especially true when we have the double
well potential. Some key properties we might care about include:
1. the density along the x direction
2. the pair correlation function between the electrons


Use the following function:


In [ ]:
def CalcDensity(self,DensityHistogram):
    for slice in range(0,self.NumTimeSlices):
        dist=numpy.sqrt(dot(self.beads[slice,0],self.beads[slice,0]))
        DensityHistogram.add(self.beads[slice,0,0])
        dist=numpy.sqrt(dot(self.beads[slice,1],self.beads[slice,1]))
        DensityHistogram.add(self.beads[slice,1,0])


Using this as inspiration, you should write the function

In [ ]:
def CalcPairCorrelation(Path,PairHistogram):
#simply stores the current pair correlation in a histogram

Remember, that to add a value to a histogram you need to call

In [ ]:
PairHistogram.add(myVal)

(Notice that in order to calculate these properties, we are going to want to sum over all
the slices in imaginary time.)
Once this function is written, we have to initialize these histograms and plot them inside
our PIMC function. You may have noticed the lines inside the PIMC function

In [ ]:
DensityHistogram=Histogram(minVal,maxVal,numPoints)

and 

In [ ]:
Histogram.PlotMe("fileName.png")

that will initialize and plot our histogram.
You should now run your simulation and plot these observables. From the density plot, it
should be clear that the particles fill out the harmonic potential effectively. You should
be able to look at the pair correlation plot and notice that the two particles are
independent.

### Correlation: The Interaction Potential

So far, everything we have done has involved independent electrons. Of course, the whole
point of doing quantum monte carlo is so you can calculate properties of correlated
electrons. So far we have been construction our Path with the coupling constant of the
interaction potential set to zero. Now we will write a function for the interaction
potential of the form

$$V_{ee}(r)=\frac{c}{r_{12}}$$

where $r_{12}$ is the distance between the two particles.  Depending on whether the coupling
constant $c$ is positive or negative will determine whether or not the system is attractive
or repulsive. Your function should call Path.c to access the coupling constant. Let's make
sure the function

In [ ]:
def Vee(self,R):
    #compute coulumb potential
    #remembering to add the epsilon
    # for attractive potentials.
    # self.C accesses the C

is filled in. We can now run the code with the repulsive potential

In [ ]:
Path.SetPotential(HarmonicOscillator)
Path.SetCouplingConstant(1.0)
PIMC(numSteps,Path,[DisplaceMove,BisectionMove],"Coupling")

How do the pair correlation functions of this function differ from the pair correlation
when there was no attraction?

### Adding the double well

At this point we are ready to add the second well to the system. In order to do this, we
will write a potential for the double harmonic well

$$V(r)= s((x-a)^2+y^2+z^2))((x+a)^2+y^2+z^2))$$

Use $a=2$ and $s=0.1$

Write a function

In [ ]:
def DoubleWell(Path,r):
# Return the double well potential
# Remember that r is a 3-vector (stored as a numpy array)

Then we can call the code with 

In [ ]:
Path.SetCouplingConstant(0.0)
print PIMC(numSteps,Path,[BisectionMove,DisplaceMove],"DoubleWellNoCoupling")
Path.SetCouplingConstant(1.0)
print PIMC(numSteps,Path,[BisectionMove,DisplaceMove],"DoubleWellRepulsive")

Now run your code and look at your density and pair correlation functions. You should
notice that the particles (when repulsive) stay in opposing wells.
Now we will try turning off the interaction potential entirely (TestDoubleWellNone.py).
What happens to the density profile and pair correlation function?

### Manipulating a quantum dot

Congratulations! You have now put together a complete path integral code for examining two
unpolarized electrons in a quantum dot. At this point, you might want to explore some
properties of your quantum dot. For example you might examine the range of coupling
constants an experimentalist might have to sample over in order to change between electrons
staying in their respective dot in comparison with electrons staying in different dots.
Beyond, this you could examine at what temperature, the coherence of these dots is
effectively diminished.